### 데이터 읽기

In [19]:
import pandas as pd

# CSV 파일 읽기
call119 = pd.read_csv('call119_train.csv')
cat119 = pd.read_csv('cat119_train.csv')
test_call119 = pd.read_csv('test_call119.csv', encoding='cp949')
test_cat119 = pd.read_csv('test_cat119.csv', encoding='cp949')
solar = pd.read_csv('solar_data.csv', encoding='cp949')


In [20]:
# 데이터 확인
print("call119_train.csv")
call119.head(10)

call119_train.csv


,Unnamed: 0,call119_train.tm,call119_train.address_city,call119_train.address_gu,call119_train.sub_address,call119_train.stn,call119_train.ta_max,call119_train.ta_min,call119_train.ta_max_min,call119_train.hm_min,call119_train.hm_max,call119_train.ws_max,call119_train.ws_ins_max,call119_train.rn_day,call119_train.call_count
0,1,20200501,부산광역시,강서구,대저2동,904,23.7,16.6,7.1,63.2,90.3,5.8,10.6,0.0,1
1,2,20200501,부산광역시,강서구,천성동,921,24.3,15.4,8.9,64.2,96.4,4.8,8.6,0.0,1
2,3,20200501,부산광역시,금정구,금사동,940,25.6,17.2,8.4,46.6,76.4,4.8,8.3,0.0,1
3,4,20200501,부산광역시,금정구,금성동,941,25.3,17.1,8.2,-99.0,-99.0,7.1,9.7,0.0,1
4,5,20200501,부산광역시,금정구,남산동,939,26.5,16.7,9.8,-99.0,-99.0,5.7,8.1,0.0,3
5,6,20200501,부산광역시,금정구,부곡동,940,25.6,17.2,8.4,46.6,76.4,4.8,8.3,0.0,1
6,7,20200501,부산광역시,금정구,장전동,940,25.6,17.2,8.4,46.6,76.4,4.8,8.3,0.0,1
7,8,20200501,부산광역시,기장군,기장읍,923,26.9,15.9,11.0,49.7,87.6,4.2,7.3,0.0,1
8,9,20200501,부산광역시,기장군,일광면,923,26.9,15.9,11.0,49.7,87.6,4.2,7.3,0.0,2
9,10,20200501,부산광역시,기장군,장안읍,923,26.9,15.9,11.0,49.7,87.6,4.2,7.3,0.0,1


In [21]:
print("cat119_train.csv")
cat119.head(10)

cat119_train.csv


,Unnamed: 0,cat119_train.tm,cat119_train.address_city,cat119_train.address_gu,cat119_train.sub_address,cat119_train.cat,cat119_train.sub_cat,cat119_train.stn,cat119_train.call_count
0,1,20200501,부산광역시,강서구,대저2동,구급,교통사고,904,1
1,2,20200501,부산광역시,강서구,천성동,구조,교통사고,921,1
2,3,20200501,부산광역시,금정구,금사동,구급,부상,940,1
3,4,20200501,부산광역시,금정구,금성동,기타,업무운행,941,1
4,5,20200501,부산광역시,금정구,남산동,구급,부상,939,1
5,6,20200501,부산광역시,금정구,남산동,기타,업무운행,939,2
6,7,20200501,부산광역시,금정구,부곡동,구급,부상,940,1
7,8,20200501,부산광역시,금정구,장전동,구급,교통사고,940,1
8,9,20200501,부산광역시,기장군,기장읍,구급,교통사고,923,1
9,10,20200501,부산광역시,기장군,일광면,구급,부상,923,1


In [22]:
print("cat119_train.csv")
solar.head(10)

cat119_train.csv


,지점,일시,가조시간(hr),합계 일조시간(hr),1시간 최다일사 시각(hhmi),1시간 최다일사량(MJ/m2),합계 일사량(MJ/m2)
0,159,2020-05-01,13.6,11.6,1200.0,3.54,26.87
1,159,2020-05-02,13.6,5.0,1000.0,2.82,17.20
2,159,2020-05-03,13.7,0.0,1200.0,0.49,3.05
3,159,2020-05-04,13.7,9.7,1200.0,3.44,24.98
4,159,2020-05-05,13.7,1.9,900.0,2.29,15.24
5,159,2020-05-06,13.8,12.3,1200.0,3.61,28.54
6,159,2020-05-07,13.8,12.2,1100.0,3.67,29.25
7,159,2020-05-08,13.8,7.2,1000.0,3.22,23.22
8,159,2020-05-09,13.9,0.0,1200.0,0.37,2.08
9,159,2020-05-10,13.9,6.1,1400.0,2.75,20.27


In [23]:
call119.tail()

,Unnamed: 0,call119_train.tm,call119_train.address_city,call119_train.address_gu,call119_train.sub_address,call119_train.stn,call119_train.ta_max,call119_train.ta_min,call119_train.ta_max_min,call119_train.hm_min,call119_train.hm_max,call119_train.ws_max,call119_train.ws_ins_max,call119_train.rn_day,call119_train.call_count
42919,42920,20231031,부산광역시,해운대구,반송동,937,26.3,8.8,17.5,20.2,97.4,4.6,8.4,0.0,2
42920,42921,20231031,부산광역시,해운대구,반여동,937,26.3,8.8,17.5,20.2,97.4,4.6,8.4,0.0,2
42921,42922,20231031,부산광역시,해운대구,석대동,940,24.5,10.0,14.5,20.4,94.0,2.4,6.1,0.0,1
42922,42923,20231031,부산광역시,해운대구,송정동,937,26.3,8.8,17.5,20.2,97.4,4.6,8.4,0.0,1
42923,42924,20231031,부산광역시,해운대구,재송동,937,26.3,8.8,17.5,20.2,97.4,4.6,8.4,0.0,1


In [24]:
cat119.tail()

,Unnamed: 0,cat119_train.tm,cat119_train.address_city,cat119_train.address_gu,cat119_train.sub_address,cat119_train.cat,cat119_train.sub_cat,cat119_train.stn,cat119_train.call_count
61766,61767,20231031,부산광역시,해운대구,반여동,구급,교통사고,937,1
61767,61768,20231031,부산광역시,해운대구,반여동,구급,부상,937,1
61768,61769,20231031,부산광역시,해운대구,석대동,구급,부상,940,1
61769,61770,20231031,부산광역시,해운대구,송정동,기타,업무운행,937,1
61770,61771,20231031,부산광역시,해운대구,재송동,구급,부상,937,1


In [25]:
solar.tail()

,지점,일시,가조시간(hr),합계 일조시간(hr),1시간 최다일사 시각(hhmi),1시간 최다일사량(MJ/m2),합계 일사량(MJ/m2)
915,159,2024-10-27,10.9,2.2,900.0,1.42,6.39
916,159,2024-10-28,10.9,2.5,900.0,1.58,6.88
917,159,2024-10-29,10.9,4.7,1000.0,2.21,11.86
918,159,2024-10-30,10.8,9.9,1100.0,2.46,15.35
919,159,2024-10-31,10.8,2.9,1300.0,1.81,10.05


In [26]:
# Info
call119.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42924 entries, 0 to 42923
Data columns (total 15 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Unnamed: 0                  42924 non-null  int64  
 1   call119_train.tm            42924 non-null  int64  
 2   call119_train.address_city  42924 non-null  object 
 3   call119_train.address_gu    42924 non-null  object 
 4   call119_train.sub_address   42924 non-null  object 
 5   call119_train.stn           42924 non-null  int64  
 6   call119_train.ta_max        42924 non-null  float64
 7   call119_train.ta_min        42924 non-null  float64
 8   call119_train.ta_max_min    42924 non-null  float64
 9   call119_train.hm_min        42924 non-null  float64
 10  call119_train.hm_max        42924 non-null  float64
 11  call119_train.ws_max        42924 non-null  float64
 12  call119_train.ws_ins_max    42924 non-null  float64
 13  call119_train.rn_day        429

In [27]:
cat119.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61771 entries, 0 to 61770
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Unnamed: 0                 61771 non-null  int64 
 1   cat119_train.tm            61771 non-null  int64 
 2   cat119_train.address_city  61771 non-null  object
 3   cat119_train.address_gu    61771 non-null  object
 4   cat119_train.sub_address   61771 non-null  object
 5   cat119_train.cat           61771 non-null  object
 6   cat119_train.sub_cat       61771 non-null  object
 7   cat119_train.stn           61771 non-null  int64 
 8   cat119_train.call_count    61771 non-null  int64 
dtypes: int64(4), object(5)
memory usage: 4.2+ MB


In [28]:
solar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 920 entries, 0 to 919
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   지점                 920 non-null    int64  
 1   일시                 920 non-null    object 
 2   가조시간(hr)           920 non-null    float64
 3   합계 일조시간(hr)        918 non-null    float64
 4   1시간 최다일사 시각(hhmi)  918 non-null    float64
 5   1시간 최다일사량(MJ/m2)   918 non-null    float64
 6   합계 일사량(MJ/m2)      918 non-null    float64
dtypes: float64(5), int64(1), object(1)
memory usage: 50.4+ KB


In [29]:
# Describe
call119.describe()

,Unnamed: 0,call119_train.tm,call119_train.stn,call119_train.ta_max,call119_train.ta_min,call119_train.ta_max_min,call119_train.hm_min,call119_train.hm_max,call119_train.ws_max,call119_train.ws_ins_max,call119_train.rn_day,call119_train.call_count
count,42924.000000,4.292400e+04,42924.000000,42924.000000,42924.000000,42924.000000,42924.000000,42924.000000,42924.000000,42924.000000,42924.000000,42924.000000
mean,21462.500000,2.021586e+07,816.492126,26.025443,18.431968,7.286723,45.600601,76.598558,4.422377,8.185700,8.280265,1.916760
std,12391.235814,1.100894e+04,278.509107,8.059514,8.093267,6.623270,43.425352,49.403930,6.343533,7.183937,27.216865,2.058855
min,1.000000,2.020050e+07,159.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,-99.000000,1.000000
25%,10731.750000,2.021051e+07,904.000000,23.600000,15.500000,5.500000,40.200000,83.300000,3.400000,6.100000,0.000000,1.000000
50%,21462.500000,2.021103e+07,938.000000,26.600000,19.500000,7.500000,56.500000,91.800000,4.200000,7.700000,0.000000,1.000000
75%,32193.250000,2.022103e+07,940.000000,29.400000,22.500000,9.600000,67.700000,96.700000,5.500000,9.900000,2.000000,2.000000
max,42924.000000,2.023103e+07,950.000000,37.600000,29.000000,23.000000,99.500000,100.000000,30.700000,37.800000,259.200000,115.000000


In [30]:
cat119.describe()

,Unnamed: 0,cat119_train.tm,cat119_train.stn,cat119_train.call_count
count,61771.000000,6.177100e+04,61771.000000,61771.000000
mean,30886.000000,2.021578e+07,835.680934,1.331936
std,17831.896076,1.086659e+04,259.170544,1.288823
min,1.000000,2.020050e+07,159.000000,1.000000
25%,15443.500000,2.021052e+07,923.000000,1.000000
50%,30886.000000,2.021102e+07,938.000000,1.000000
75%,46328.500000,2.022102e+07,941.000000,1.000000
max,61771.000000,2.023103e+07,950.000000,72.000000


In [31]:
solar.describe()

,지점,가조시간(hr),합계 일조시간(hr),1시간 최다일사 시각(hhmi),1시간 최다일사량(MJ/m2),합계 일사량(MJ/m2)
count,920.0,920.000000,918.000000,918.000000,918.000000,918.000000
mean,159.0,13.317826,6.220370,1191.067538,2.639532,17.854139
std,0.0,1.156092,4.329407,123.148242,0.937018,8.040531
min,159.0,10.800000,0.000000,800.000000,0.210000,1.290000
25%,159.0,12.400000,1.900000,1100.000000,2.120000,12.067500
50%,159.0,13.800000,6.850000,1200.000000,2.915000,18.500000
75%,159.0,14.300000,10.100000,1200.000000,3.400000,24.965000
max,159.0,14.500000,12.600000,1600.000000,3.910000,30.730000


### 컬럼명 정리

In [32]:
call119.columns = [col.replace('call119_train.', '') for col in call119.columns]
test_call119.columns = test_call119.columns.str.lower()

test_call119.columns = [col.upper() if col == 'tm' else col for col in test_call119.columns]


In [33]:
# WBGT 함수
def calculate_wbgt(ta_max, ta_min, rh_max, rh_min, ws_ins_max, avg_solar):
    t_avg = (ta_max + ta_min) / 2
    rh_avg = (rh_max + rh_min) / 2
    e = rh_avg / 100 * 6.105 * np.exp((17.27 * t_avg) / (237.7 + t_avg))
    wbgt_base = 0.567 * t_avg + 0.393 * e + 3.94
    alpha = 1.0
    beta = 0.5
    solar_factor = (avg_solar / 800) if avg_solar is not None else 0
    wind = ws_ins_max if ws_ins_max is not None else 0
    return wbgt_base + alpha * solar_factor - beta * wind

In [34]:
call119['TM'] = pd.to_datetime(call119['tm'], format='%Y%m%d')
cat119['TM'] = pd.to_datetime(cat119['cat119_train.tm'], format='%Y%m%d')
solar['TM'] = pd.to_datetime(solar['일시'], format='%Y-%m-%d')
solar['avg_solar'] = solar['합계 일사량(MJ/m2)'] * 11.574
solar = solar[['TM', '지점', 'avg_solar']].rename(columns={'지점': 'stn'})
test_call119['TM'] = pd.to_datetime(test_call119['TM'], format='%Y%m%d')
test_cat119['TM'] = pd.to_datetime(test_call119['TM'], format='%Y%m%d')

In [ ]:
solar.columns

Index(['TM', 'stn', 'avg_solar'], dtype='object')

In [50]:
call119.columns


Index(['Unnamed: 0', 'tm', 'address_city', 'address_gu', 'sub_address', 'stn',
       'ta_max', 'ta_min', 'ta_max_min', 'hm_min', 'hm_max', 'ws_max',
       'ws_ins_max', 'rn_day', 'call_count', 'TM', 'avg_solar'],
      dtype='object')

In [51]:
cat119 = cat119.rename(columns=lambda x: x.replace('cat119_train.', '') if x.startswith('cat119_train.') else x)
cat119.columns

Index(['Unnamed: 0', 'tm', 'address_city', 'address_gu', 'sub_address', 'cat',
       'sub_cat', 'stn', 'call_count', 'TM'],
      dtype='object')

In [52]:
test_call119.columns

Index(['TM', 'address_city', 'address_gu', 'sub_address', 'stn', 'ta_max',
       'ta_min', 'ta_max_min', 'hm_min', 'hm_max', 'ws_max', 'ws_ins_max',
       'rn_day', 'call_count', 'avg_solar'],
      dtype='object')

In [54]:
# 병합 전 데이터타입 확인
print(test_call119['TM'].dtype)
print(solar['TM'].dtype)
print(test_cat119['TM'].dtype)

datetime64[ns]
datetime64[ns]
datetime64[ns]


In [55]:
solar['avg_solar'] = solar['avg_solar'] * 11.574
solar = solar[['TM', 'stn', 'avg_solar']]
solar = solar.rename(columns={'지점': 'stn'})

# call119 데이터 병합
call119 = call119.merge(solar, left_on=['TM', 'stn'], right_on=['TM', 'stn'], how='left')
test_call119 = test_call119.merge(solar, left_on=['TM', 'stn'], right_on=['TM', 'stn'], how='left')

In [57]:
import numpy as np
call119['WBGT'] = calculate_wbgt(call119['ta_max'], call119['ta_min'], call119['hm_max'], call119['hm_min'], call119['ws_ins_max'], call119['avg_solar'])
test_call119['WBGT'] = calculate_wbgt(test_call119['ta_max'], test_call119['ta_min'], test_call119['hm_max'], test_call119['hm_min'], test_call119['ws_ins_max'], test_call119['avg_solar'])

In [60]:
# 경로확인
# import sys
# print(sys.executable)

In [61]:
from sklearn.model_selection import train_test_split
import xgboost as xgb

features = ['ta_max', 'ta_min', 'ta_max_min', 'hm_max', 'hm_min', 'ws_max', 'ws_ins_max', 'rn_day', 'WBGT']
target = 'call_count'

X = call119[features]
y = call119[target]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

model = xgb.XGBRegressor(n_estimators=100, max_depth=4, learning_rate=0.1)
model.fit(X_train, y_train)

# 예측
val_pred = model.predict(X_val)

In [64]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

r2 = r2_score(y_val, val_pred)
rmse = np.sqrt(mean_squared_error(y_val, val_pred))

print(f'R² Score: {r2:.4f}')
print(f'RMSE: {rmse:.4f}')

R² Score: 0.2041
RMSE: 1.6696


In [41]:
merged = pd.merge(call119, solar, on='TM', how='inner')

In [42]:
merged.head()

,Unnamed: 0,tm,address_city,address_gu,sub_address,stn_x,ta_max,ta_min,ta_max_min,hm_min,hm_max,ws_max,ws_ins_max,rn_day,call_count,TM,avg_solar_x,stn_y,avg_solar_y
0,1,20200501,부산광역시,강서구,대저2동,904,23.7,16.6,7.1,63.2,90.3,5.8,10.6,0.0,1,2020-05-01,NaN,159,3599.43738
1,2,20200501,부산광역시,강서구,천성동,921,24.3,15.4,8.9,64.2,96.4,4.8,8.6,0.0,1,2020-05-01,NaN,159,3599.43738
2,3,20200501,부산광역시,금정구,금사동,940,25.6,17.2,8.4,46.6,76.4,4.8,8.3,0.0,1,2020-05-01,NaN,159,3599.43738
3,4,20200501,부산광역시,금정구,금성동,941,25.3,17.1,8.2,-99.0,-99.0,7.1,9.7,0.0,1,2020-05-01,NaN,159,3599.43738
4,5,20200501,부산광역시,금정구,남산동,939,26.5,16.7,9.8,-99.0,-99.0,5.7,8.1,0.0,3,2020-05-01,NaN,159,3599.43738


### 함수 선언 전에 컬럼 정리한 후 다시 ㄱㄱ

In [43]:
# def merge_and_predict(call119, solar):
#     import pandas as pd
#     from sklearn.model_selection import train_test_split
#     from sklearn.linear_model import LinearRegression
#     from sklearn.metrics import mean_squared_error, r2_score

#     # 컬럼명 정리
#     call119 = call119.rename(columns=lambda x: x.replace('call119_train.', '') if x.startswith('call119_train.') else x)

#     # 날짜 타입 변환
#     if 'TM' not in call119.columns:
#         call119['TM'] = pd.to_datetime(call119['tm'].astype(str), format='%Y%m%d')
#     else:
#         call119['TM'] = pd.to_datetime(call119['TM'])
#     solar['TM'] = pd.to_datetime(solar['TM'])

#     # 병합
#     merged = call119.merge(solar[['TM', 'avg_solar']], on='TM', how='left')

#     # 범주형 컬럼 자동 인코딩 (문자열 전부)
#     cat_cols = merged.select_dtypes(include=['object']).columns.tolist()
#     if 'TM' in cat_cols:
#         cat_cols.remove('TM')
#     merged = pd.get_dummies(merged, columns=cat_cols, drop_first=True)

#     # 결측치 제거(특히 타겟)
#     merged = merged.dropna(subset=['call_count'])

#     # 숫자형 피처만 추출
#     feature_cols = merged.select_dtypes(include=['float64', 'int64']).columns.tolist()
#     feature_cols = [c for c in feature_cols if c not in ['call_count', 'TM', 'stn', 'tm']]

#     # 결측치 보간
#     merged[feature_cols] = merged[feature_cols].interpolate(method='linear', limit_direction='both')

#     X = merged[feature_cols]
#     y = merged['call_count']

#     # 학습/평가
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#     model = LinearRegression()
#     model.fit(X_train, y_train)

#     y_pred = model.predict(X_test)
#     rmse = mean_squared_error(y_test, y_pred, squared=False)
#     r2 = r2_score(y_test, y_pred)

#     print(f"✅ RMSE: {rmse:.4f}")
#     print(f"✅ R^2: {r2:.4f}")

#     return merged, model, feature_cols


In [44]:
def merge_and_predict(call119, solar):
    import pandas as pd
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_squared_error, r2_score
    import numpy as np

    # 컬럼명 정리
    call119 = call119.rename(columns=lambda x: x.replace('call119_train.', '') if x.startswith('call119_train.') else x)

    # 날짜 타입 변환
    if 'TM' not in call119.columns:
        call119['TM'] = pd.to_datetime(call119['tm'].astype(str), format='%Y%m%d')
    else:
        call119['TM'] = pd.to_datetime(call119['TM'])
    solar['TM'] = pd.to_datetime(solar['TM'])

    # 병합
    merged = call119.merge(solar[['TM', 'avg_solar']], on='TM', how='left')

    # 범주형 컬럼 자동 인코딩 (문자열 전부)
    cat_cols = merged.select_dtypes(include=['object']).columns.tolist()
    if 'TM' in cat_cols:
        cat_cols.remove('TM')
    merged = pd.get_dummies(merged, columns=cat_cols, drop_first=True)

    # 결측치 제거(특히 타겟)
    merged = merged.dropna(subset=['call_count'])

    # 숫자형 피처만 추출
    feature_cols = merged.select_dtypes(include=['float64', 'int64']).columns.tolist()
    feature_cols = [c for c in feature_cols if c not in ['call_count', 'TM', 'stn', 'tm']]

    # 결측치 보간
    merged[feature_cols] = merged[feature_cols].interpolate(method='linear', limit_direction='both')

    X = merged[feature_cols]
    y = merged['call_count']

    # 학습/평가
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = LinearRegression()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    print(f"✅ RMSE: {rmse:.4f}")
    print(f"✅ R^2: {r2:.4f}")

    return merged, model, feature_cols

In [45]:
# 병합 전 데이터타입 변환
if test_call119['TM'].dtype != 'datetime64[ns]':
    test_call119['TM'] = pd.to_datetime(test_call119['TM'].astype(str), format='%Y%m%d')

In [46]:
# def preprocess_and_predict_test(test_call119, solar, model, train_columns):
#     import pandas as pd

#     # 1. 날짜 처리
#     test_call119['TM'] = pd.to_datetime(test_call119['TM'].astype(str), format='%Y%m%d')
#     solar['TM'] = pd.to_datetime(solar['TM'])

#     # 2. 컬럼명 정리
#     test_call119 = test_call119.rename(columns={
#         'stn': 'STN',
#         'ta_max': 'ta_max',
#         'ta_min': 'ta_min',
#         'ta_max_min': 'ta_max_min',
#         'hm_min': 'hm_min',
#         'hm_max': 'hm_max',
#         'ws_max': 'ws_max',
#         'ws_ins_max': 'ws_ins_max',
#         'rn_day': 'rn_day',
#         'sub_address': 'sub_address'
#     })

#     # 3. 병합
#     test_merged = test_call119.merge(solar, on=['TM', 'stn'], how='left')

#     # 4. 지역 더미 변수 (prefix 없이)
#     test_merged = pd.get_dummies(test_merged, columns=['sub_address'], drop_first=True)

#     # 5. 누락된 학습 컬럼 보정
#     for col in train_columns:
#         if col not in test_merged.columns:
#             test_merged[col] = 0

#     # 6. 컬럼 순서 정렬
#     test_merged = test_merged[train_columns]

#     # 7. 결측치 보간
#     numeric_cols = test_merged.select_dtypes(include=['float64', 'int64']).columns
#     test_merged[numeric_cols] = test_merged[numeric_cols].interpolate(method='linear', limit_direction='both')

#     # 8. 예측
#     y_pred = model.predict(test_merged)

#     return y_pred

In [47]:
def preprocess_and_predict_test(test_call119, solar, model, train_columns):
    import pandas as pd

    # 1. 날짜 처리 (이미 TM 컬럼이 있다고 가정)
    test_call119['TM'] = pd.to_datetime(test_call119['TM'])
    solar['TM'] = pd.to_datetime(solar['TM'])

    # 2. 병합 (STN 제거, TM만으로 병합)
    test_merged = test_call119.merge(solar, on='TM', how='left')

    # 3. 지역 더미 변수
    test_merged = pd.get_dummies(test_merged, columns=['sub_address'], drop_first=True)

    # 4. 학습시 사용된 컬럼과 동일하게 맞추기 (누락 컬럼 0으로 채움)
    for col in train_columns:
        if col not in test_merged.columns:
            test_merged[col] = 0

    # 5. 컬럼 순서 맞추기
    test_merged = test_merged[train_columns]

    # 6. 결측치 보간
    numeric_cols = test_merged.select_dtypes(include=['float64', 'int64']).columns
    test_merged[numeric_cols] = test_merged[numeric_cols].interpolate(method='linear', limit_direction='both')

    # 7. 예측
    y_pred = model.predict(test_merged)

    return y_pred

In [48]:
# 병합 후
merged = call119.merge(solar[['TM', 'avg_solar']], on='TM', how='left')

# 자동으로 범주형 컬럼 감지 및 원-핫 인코딩
cat_cols = merged.select_dtypes(include=['object']).columns.tolist()
if 'TM' in cat_cols:
    cat_cols.remove('TM')
merged = pd.get_dummies(merged, columns=cat_cols, drop_first=True)

In [49]:
# 학습
merged_train, model, feature_cols = merge_and_predict(call119, solar)

# 예측에 필요한 feature 컬럼 저장
feature_cols = list(merged_train.drop(columns=['call_count', 'TM']).columns)

# 테스트셋 예측
y_test_pred = preprocess_and_predict_test(test_call119, solar, model, feature_cols)

# 예측 결과 확인
print("📈 예측 결과 상위 5개:")
print(y_test_pred[:5])


✅ RMSE: 1.8409
✅ R^2: 0.0325


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- address_gu_금정구
- address_gu_기장군
- address_gu_남구
- address_gu_동구
- address_gu_동래구
- ...


In [ ]:
test_call119.columns

In [ ]:
# # 지역 변수 원핫 인코딩 (sub_address)
# call119 = pd.get_dummies(call119, columns=['call119_train.sub_address'], prefix='sub', drop_first=True)
# test_call119 = pd.get_dummies(test_call119, columns=['sub_address'], prefix='sub', drop_first=True)

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from sklearn.model_selection import train_test_split
# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error
# from sklearn.preprocessing import LabelEncoder

In [ ]:
# import matplotlib.font_manager as fm
# # 예: 맑은 고딕 폰트 사용
# plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# # plt.rcParams['font.family'] = 'AppleGothic'  # macOS
# # 마이너스 깨짐 방지
# plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# # 파생 변수 생성 (WBGT + 일조율)
# def calculate_wbgt(ta_max, ta_min, rh_max, rh_min, ws_ins_max, avg_solar):
#     # 1. 평균 기온과 습도
#     t_avg = (ta_max + ta_min) / 2
#     rh_avg = (rh_max + rh_min) / 2

#     # 2. 수증기압 (hPa)
#     e = rh_avg / 100 * 6.105 * np.exp((17.27 * t_avg) / (237.7 + t_avg))

#     # 3. 기본 WBGT 계산 (Steadman 근사식)
#     wbgt_base = 0.567 * t_avg + 0.393 * e + 3.94

#     # 4. 일사량과 풍속 보정
#     alpha = 1.0  # 일사량 영향 계수
#     beta = 0.5   # 풍속 영향 계수
#     solar_factor = (avg_solar / 800) if avg_solar is not None else 0
#     wind = ws_ins_max if ws_ins_max is not None else 0

#     wbgt_adjusted = wbgt_base + alpha * solar_factor - beta * wind
#     return wbgt_adjusted

In [ ]:
# EDA 시각화 예시
sns.boxplot(data=merged, x='tm_weekday', y='call119_train.call_count')
plt.title('요일별 119 신고건수')
plt.show()

sns.scatterplot(data=merged, x='WBGT', y='call119_train.call_count')
plt.title('WBGT vs 신고건수')
plt.show()

In [ ]:
# 신고 유형별 집계
cat_grouped = cat119_df.groupby(['tm', 'cat119_train.cat'])['cat119_train.call_count'].sum().reset_index()
cat_grouped = cat_grouped.pivot(index='tm', columns='cat119_train.cat', values='cat119_train.call_count').fillna(0).reset_index()
cat_grouped.head()


In [ ]:
# 날짜 기준 병합
final = pd.merge(merged, cat_grouped, left_on='tm', right_on='tm', how='left')
final.head()

In [ ]:
# 모델 학습
features = [
    'call119_train.ta_max', 'call119_train.ta_min', '일교차',
    'call119_train.hm_max', 'call119_train.hm_min',
    'call119_train.ws_max', 'call119_train.ws_ins_max',
    'call119_train.rn_day', '합계 일사량(MJ/m2)', '합계 일조시간(hr)',
    '가조시간(hr)', '일조율', 'WBGT', 'tm_weekday', 'tm_month'
]


In [ ]:
# 타겟: '구급' 신고 예측 (예: 폭염 관련성 높음)
y = final['구급']
X = final[features].fillna(0)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=200, random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_test)

# RMSE 직접 계산
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("✅ RMSE (구급 신고 예측):", rmse)

In [ ]:
from sklearn.metrics import r2_score

r2 = r2_score(y_test, pred)
print("📈 R² score:", r2)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# 기본 지표들
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, pred)
r2 = r2_score(y_test, pred)

# 상대 지표들
mean_actual = np.mean(y_test)
max_actual = np.max(y_test)
min_actual = np.min(y_test)

# 상대적 오차 비율 (백분율)
rmse_pct_mean = (rmse / mean_actual) * 100
mae_pct_mean = (mae / mean_actual) * 100

print("📊 평가 지표 요약")
print(f"✅ RMSE: {rmse:.3f}")
print(f"✅ MAE: {mae:.3f}")
print(f"📈 R² score: {r2:.4f}")
print()
print(f"📌 평균 실제값: {mean_actual:.3f}")
print(f"📌 최대 실제값: {max_actual}")
print()
print(f"📉 RMSE / 평균값: {rmse_pct_mean:.2f}%")
print(f"📉 MAE / 평균값: {mae_pct_mean:.2f}%")

In [ ]:
#MAE가 지나치게 작고, RMSE는 상대적으로 크다 → 분산이 큰 몇 개의 극단값(outlier) 때문에 RMSE가 영향을 크게 받은 것일 수 있음


In [ ]:
import matplotlib.font_manager as fm
# 예: 맑은 고딕 폰트 사용
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # macOS
# 마이너스 깨짐 방지
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(y_test, pred, alpha=0.7)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
plt.xlabel('실제값 (y_test)')
plt.ylabel('예측값 (pred)')
plt.title('📈 실제값 vs 예측값')
plt.grid(True)
plt.show()

In [ ]:
# 이상치(극단값)는 거의 없음 → 좋은 모델입니다.

# RMSE와 MAE의 차이는 자연스러운 현상이며, 예측이 어려운 일부 사례가 RMSE에 조금 더 영향을 준 정도입니다.

# R² = 0.9956, RMSE / 평균 = 1.43% → 실제로는 매우 안정적이고 정확한 모델이므로 걱정할 필요는 전혀 없습니다.